In [1]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import networkx as nx

In [2]:
def css_styling():
    styles = open('../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

**Question** Write your name here

<div class='response'>
Answer here
</div>

**Question** Write your partner's name here

<div class='response'>
Answer here
</div>

**Question** Write your partner's favorite book here

<div class='response'>
Answer here
</div>

# Lab 05

## The Erdos-Renyi random graph model

Last week, we learned about several different ways to measure network connectivity. This week, we're going to take our first look at a theoretical model of a network: the [Erdos-Renyi random graph model](https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model). This model is sometimes also called the ER model. (Note: technically, there are a few different versions of the ER model, but they are qualitatively very similar to each other. We're not going to dwell on all of the different versions here.)

One way to think about the ER model is that it is a recipe that describes how networks can be generated. The ER model has two parameters

* $n$ - the number of nodes in the network
* $p$ - for each pair of nodes, the probability that those nodes are connected by an edge

You can think of the parameter $p$ as being a single number from $0$ to $1$ that controls the level of connectedness of the network. When $p=0$, the network is not connected at all; and when $p=1$, the network is completely connected (all possible edges exist). We are usually interested in $0 \lt p \lt 1$, ie, intermediate values of $p$.

To generate an ER graph for particular parameter values $n$ and $p$, you could follow this algorithm:

    for each pair of vertices u and v:
        flip a coin that shows heads with probability p
        if the coin is heads:
            add the edge (u,v) to the network
        if the coin is tails:
            do nothing

[BOARD DEMONSTRATION OF ER ALGORITHM]

One way to understand the behavior of the ER model is to analyze it mathematically; in fact, this is what Erdos and Renyi originally did. It turns out that this simple model has some rich and interesting behavior.

In this class, we're going to study the ER model by using simulation; in other words, we're going to use the computer to create lots of different random networks in order to try to better understand the ER model. Simulation can be a powerful tool for answering all sorts of questions about network models.

### Investigating variation with fixed parameters

The ER model is *stochastic*; in other words, it describes a process that has some randomness. If we pick a particular value for $n$ and $p$ and use the ER model to create a network, we do not know for certain exactly which network we will get.

Let's illustrate this with a very simple example: $n = 2$. In this case, there are only two possible networks: Network A, which has no edges, and Network B, which has one edge (see below).

<img src="twonode_networks.png">

How likely is it that an ER model with $n=2$ will produce Network A as opposed to Network B? That depends on the parameter $p$.

**Question** Run the next code cell a few times to convince yourself that the `erdos_renyi_graph` function can produce different results, even if the values of $n$ and $p$ do not change.

In [ ]:
g = nx.erdos_renyi_graph(n=2, p=.1)
nx.draw(g)

Playing with the code cell illustrates that the model has randomness in it.  For the rest of the lab, we're going to study the output of the ER model to try and learn about systematic patterns that emerge in random ER results.

**Question** Imagine that you generated 1000 networks using the ER model with $n=2$ nodes and edge probability $p=0.1$.  (In other words, imagine that you ran the code cell above 1000 times.)  Try to predict the approximate fraction of the time you expect an edge to be included. In other words, how many out of those 1000 draws do you think will be Network B? Why did you predict this number?

<div class='response'>
Respond here
</div>

**Question** The next code cell actually simulates 1000 ER networks with $n=2$ and $p=0.1$.  Fill in the missing parts of the simulation below and then run it.

In [ ]:
num_sims = 1000
num_edges = make_array()

for i in range(...):
    sim_network = nx.erdos_renyi_graph(n=2, p=.1)
    num_edges = np.append(num_edges, ...) # here, you want to count the number of edges in the simulated network
    
print("Fraction of the time an edge was included: ", np.mean(num_edges))


**Question** How close was the result from the simulation to the number you predicted? Can you think of any way that to make the result from the simulation closer to the number you predicted?

<div class='response'>
Answer here
</div>

### Average degree with fixed parameters

OK, so we see that when we generate networks using the ER model, there will be some random variation even when $n$ and $p$ are fixed. Now we will see how much variation there is in the degrees of nodes in an ER networks when $n$ and $p$ are fixed.

First, we're going to take some code that we wrote in Lab 03 and turn it into a function. We're doing this because it will be helpful to use this code over and over again in the rest of this lab. 

Look at the following function and then look at the example of how to use it.

In [ ]:
def get_degrees(net):
    """
    Given a networkx network object, return a Table that has the
    degrees of all of the nodes in the network in a column called 'degree'
    """
    net_degrees = net.degree().values()
    return(Table().with_column('degree', net_degrees))

In [ ]:
## example usage
sim_network = nx.erdos_renyi_graph(n=100, p=.1)
degs = get_degrees(sim_network)
degs

The `get_degrees` function extracts the degrees of all of the nodes in a network and puts them in a Table. We can then make a histogram of the degrees.

In [ ]:
degs.hist('degree')

Now that we can easily make a histogram of a the degrees of a network's nodes, let us look at the degrees of a few ER networks generated with $n=1000$ and $p=0.1$.

**Question** Fill in the code below so that it prints out the degrees of nodes in 5 ER networks with $n=1000$ and $p=0.1$.

In [ ]:
num_sims = 5
g_n = 1000
g_p = 0.1

for i in range(...):
    sim_network = nx.erdos_renyi_graph(n=..., p=...)
    sim_network_degrees = ... # use the function we wrote above
    plt.figure(...)
    


Hopefully, we can see in the histograms above that the results are random, but that they are not completely haphazard: in fact, you can prove that the degree histogram, though random, has a characteristic shape called a *Poisson distribution*. Proving this fact goes beyond the scope of our class, but the important thing to understand is that when parameters are fixed, specific ER networks are random, but there is actually quite a bit of structure to the randomness.

We're going to illustrate one final point about using simulations to study ER networks with fixed $n$ and $p$.
Recall that the *average degree* of a network is the average number of connections across all of the nodes in a network. Although the average degree of a specific network generated from the ER model is a random quantity, one can prove that it will be close to the value 

$$\bar{d} = (n-1)p$$

where $\bar{d}$ is the average degree. Roughly, you can see why this is true by thinking about a typical node in the network. Each node has a possible connection to $n-1$ other nodes, and each of those possible connections will get placed in the network with probability $p$. (We won't dwell on this derivation, so if this isn't completely clear, don't worry.)

So, for a network with $n=100$ nodes and $p=0.1$, the average degree $\bar{d}$ should be close to $99 \times 0.1 = 9.9$.

The helpful thing about simulations is that we can estimate quantities like $\bar{d}$ even if we haven't been able to derive a formula for them. How? We can 

1. generate many ER networks using the fixed parameters $p$ and $n$
2. calculate whatever we are interested in for each of the generated networks
3. average the calculated quantites across all of the generated networks

It turns out that this approach can be used to estimate many quantities. We'll see a demonstration of this now.

First, we'll define a quick function that will help us calculate the average degree of a given network:

In [ ]:
def avg_degree(net):
    """
    Calculate the average degree for a networkx network
    """
    return(2.0*net.number_of_edges()/net.number_of_nodes())

And then we'll run a simulation to estimate the average degree for ER networks with $n = 100$ and $p=0.1$. (Remember from above that the formula tells us that this should be close to 9.9).

**Question** What would we expect the average degree for ER networks with $n=100$ and $p=0.1$ to be? (Use the formula above.)

<div class='response'>
[Answer here]
</div>

**Question** Fill in the missing code and run the following simulation a few times and note how close the estimate is to the value from the formula.

In [ ]:
num_sims = 100
g_n = 100
g_p = 0.1
avg_degree_results = make_array()

for i in range(...):
    sim_network = nx.erdos_renyi_graph(n=g_n, p=g_p)
    sim_network_avg_degree = ... # use the avg_degree function here
    ... = np.append(..., sim_network_avg_degree)
    
    
print("Estimated average degree: ", np.mean(avg_degree_results))

**Question** If we want to make the estimate more accurate, we can simply increase the number of simulations that we run. Copy the code above to the cell below, and then change `num_sims` to produce a more accurate estimate.

In [ ]:
...
...
...

Here are some general conclusions to think about from this part of the lab:

* Even for fixed parameter values, the ER model generates networks randomly, i.e., it does not produce the same network every time
* ER networks are random, but there is still a lot of structure to them -- in other words, the range of possible networks the ER model can produce is relatively narrow when compared to all possible networks.
* We can estimate some properties of ER networks by using simulation, and the more simulations we conduct, the more accurate our estimates will be

### Investigating variation across different values of $p$

In order to understand more about connectivity of networks generated by the ER model, we will investigate how the average degree of nodes in the network changes as the connectivity parameter $p$ changes.

First, let's make some specific predictions. Remember that when $p=0$, there are no edges in the network; and when $p=1$, every possible edge is in the network.

**Question** If a network has 100 nodes, what do you expect the average degree of nodes in the network to be when $p=0$? How about when $p=1$?<BR>

<div class='response'>
Respond here
</div>

**Question** How do you think the average degree will change in ER networks with $n=100$ as $p$ varies starting from $0$, then increasing until $p=1$? For example, do you think that the average degree will increase, decrease, or not change? If you think that the average degree will change, can you say anything about the shape (eg, will it be a straight line? exponential?)

<div class='response'>
Respond here
</div>

Now let's use simulation to try and get an understanding of how the average degree changes as the parameter $p$ does.

**Question** Fill in the missing parts of the simulation below, and then run it to see the results.

In [ ]:
n = 100
p_vals = np.arange(..., ..., ...) # p should go from 0 to 1 in steps of size .1
avg_degree_results = make_array()

for current_p in ...:
    sim_network = nx.erdos_renyi_graph(n=n, p=...)
    sim_network_avg_degree = ... # using the avg_degree function we wrote above
    avg_degree_results = np.append(avg_degree_results, sim_network_avg_degree)
    
results = Table().with_columns([
        'p', p_vals,
        'avg_degree', avg_degree_results
    ])

results.plot('p', 'avg_degree')

**Question** How would you describe the way that the average degree changes as $p$ does? Was your prediction correct?

<div class='response'>
Respond here
</div>

**Question** Now copy the simulation above and paste it below. Change the simulation so that $n=10$ instead of $n=100$. Run it and look at the results.

In [ ]:
...
...
...

In [ ]:
results.plot('p', 'avg_degree')

**Question** What do you see here? How does it differ from the simulation where $n=100$? Can you think of some way to explain any differences between them?

<div class='response'>
Answer here
</div>

### Optional challenge: giant components

Above, we examined how the average degree of a node changes when $n$ is fixed and $p$ changes.
In this **optional challenge** section, you will explore another aspect of network connectivity: how many 'pieces' or components there are in the network. Remember from last class that a relatively un-connected network has many different components (we called them 'islands'); a relatively connected network, on the other hand, is made up of one big island.

First, we'll re-use the `get_biggest_component` function, which we wrote together in Lab 04.

In [ ]:
def get_biggest_component(network):
    biggest = max(nx.connected_component_subgraphs(network), key=len)
    return(biggest)

We'll also add another function, `frac_nodes_in_biggest_component`, which takes a network and returns the proportion of nodes in the network that are in the biggest of the components. This quantity is an indicator of connectedness: when all of the networks nodes are part of the same giant component, then this function will return 1; on the other hand, if the network is made up of many small, separate components then this function will return a number very close to 0.

In [ ]:
def frac_nodes_in_biggest_component(net):
    gc = get_biggest_component(net)
    return(gc.number_of_nodes() / net.number_of_nodes())

Now we will use simulation to explore how the fraction of nodes in the biggest component changes as $p$ goes from smaller to larger values.

**Question** How do you expect the fraction of nodes in the biggest components to change as $p$ is varied from smaller to larger values? What shape do you think this change will take? (Of course, it is OK if your prediction ends up being wrong.)

<div class='response'>
Answer here
</div>

**Question** Now fill in the code below, which uses simulation to study this question empirically. Note that there is a new feature in this simulation: each parameter set (i.e., value of $n$ and $p$) is repeated more than once. This is accomplished using `num_repetitions` and `np.repeat`. Repeating the parameter set multiple times gives us a way to account for the fact that there is randomness to the ER networks that get generated. This will be clearer once you plot the results of the simulation below.

In [ ]:
n = 100
num_repetitions = 5
p_vals = np.repeat(np.arange(0, 0.2, .01), num_repetitions)
num_component_results = make_array()
gc_frac_results = make_array()

for current_p in ...:
    sim_network = nx.erdos_renyi_graph(n=..., p=...)
    sim_network_num_components = nx.number_connected_components(sim_network)
    sim_network_gc_frac = ... # use the frac_nodes_in_biggest_component function
    num_component_results = np.append(num_component_results, sim_network_num_components)
    gc_frac_results = np.append(gc_frac_results, sim_network_gc_frac)
    
results = Table().with_columns([
        'p', p_vals,
        'num_components', num_component_results,
        'gc_frac', gc_frac_results
    ])

**Question** Now make two scatter plots:

* one scatter plot that compares $p$ (on the x axis) to the number of components in the network (y axis)
* one scatter plot that compares $p$ (on the x axis) to the fraction of nodes in the giant component (x axis)

In [ ]:
...

In [ ]:
...

There are a couple of interesting things to notice about these plots. We'll focus on the plot that has the fraction of nodes in the giant component on the y axis (but you can study the other plot in much the same way).

First, you can tell that for some values of $p$, the result varies somewhat due to the randomness of the ER model. Given what we studied in the first part of the lab, this is not surprising. But it is important to remember this fact whenever you conduct a simulation study.

Second, looking at the plot, as $p$ increases there seems to be a nonlinear change (i.e., a change that doesn't follow a straight line) from a regime where `gc_frac` is steadily increasing to a regime where it is fixed at its maximum value, 1. When `gc_frac` is 1, all of the nodes in the network are in the same component. So this result suggests that, when $p$ is big enough, an ER random network is very likely to be completely connected, i.e., made of one 'giant' component.

Let's see if we can figure out where that transition point happens. Copy and paste the simulation code from the previous question below. Then tweak the ranges of $p$ that get explored to try and narrow down exactly where the emergence of a single giant component happens. In other words, try to find the value of $p$ above which the fraction of nodes in the giant component is equal to 1.

*[Hint: in addition to narrowing the range of $p$ values explored in the simulation, you may also want to make the distance between the different $p$ values narrower; in other words, you might want to make the third argument of `np.arange` smaller]*

In [ ]:
...
...
...

**Question** Based on your simulation exploration, what is your best estimate for the value of $p$ where a giant component emerges? Can you think of any other way of estimating this value (apart from your simulation)?

<div class='response'>
Answer here
</div>

## Survey

Please fill out [this quick survey](https://goo.gl/forms/Lmbr8o9RTDgR5Utt2) to help me understand how the class is going so far. Thanks!

## Submit the lab

You're almost done! Now please create a pdf version of your completed lab by **either**:

* printing your notebook to a pdf file
* going to the Jupyter 'File' menu, choosing 'Download as' and then 'PDF via LaTeX (.pdf)'. 

Please save the resulting .pdf on your computer and then **submit the .pdf on bcourses**.

**The lab must be submitted by the end of the day on Monday, Oct. 9. Late labs will not be accepted.**